In [1]:
import os
import numpy as np
import cv2 
from glob import glob
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [2]:
path_save_images = r'E:\GitHub\AutoDoc\NN_2\DataSetGenerator\DataSet\images'
path_save_masks = r'E:\GitHub\AutoDoc\NN_2\DataSetGenerator\DataSet\masks'

In [3]:
path_train = r'E:\GitHub\AutoDoc\NN_2\DataSetGenerator\DataSet\\'

In [4]:
def load_data():
    images = sorted([os.path.join(path_save_images, path) for path in os.listdir(path_save_images)], key = lambda a: int(a.split('\\')[-1].split('_')[0]))
    masks = sorted([os.path.join(path_save_masks, path) for path in os.listdir(path_save_masks)], key = lambda a: int(a.split('\\')[-1].split('_')[0]))

    train_x,valid_x, test_x   = np.split(images, [int(.75*len(images)), int(.95*len(images))])
    train_y,valid_y, test_y  = np.split(masks, [int(.75*len(masks)), int(.95*len(masks))])
    
    return (np.concatenate([train_x, test_x], axis=0), np.concatenate([train_y, test_y], axis=0)), (valid_x, valid_y)

In [5]:
(train_x, train_y), (valid_x, valid_y) = load_data()

In [6]:
def read_image(path):
     path = path.decode()
     x = cv2.imread(path, cv2.IMREAD_COLOR)
     x = cv2.resize(x, (256, 256))
     x = x/255.0
     return x

In [7]:
def read_mask(path):
     path = path.decode()
     x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
     x = cv2.resize(x, (256, 256))
     x = x/255.0
     x = np.expand_dims(x, axis=-1)
     return x

In [8]:
def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float64, tf.float64])
    x.set_shape([256, 256, 3])
    y.set_shape([256, 256, 1])
    return x, y

def tf_dataset(x, y, batch):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset.shuffle(buffer_size = batch)
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    
    dataset = dataset.repeat()
    return dataset

In [9]:
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model

In [10]:
def conv_block(x, num_filters):
    x = Conv2D(num_filters, (3, 3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, (3, 3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

In [22]:
def build_model():
    size = 256
    num_filters = [16, 32, 48, 64]
    inputs = Input((size, size, 3))

    skip_x = []
    x = inputs

    ## Encoder
    for f in num_filters:
        x = conv_block(x, f)
        skip_x.append(x)
        x = MaxPool2D((2, 2))(x)

    ## Bridge
    x = conv_block(x, num_filters[-1])

    num_filters.reverse()
    skip_x.reverse()

    ## Decoder
    for i, f in enumerate(num_filters):
        x = UpSampling2D((2, 2))(x)
        xs = skip_x[i]
        x = Concatenate()([x, xs])
        x = conv_block(x, f)

    ## Output
    x = Conv2D(1, (1, 1), padding="same")(x)
    x = Activation("sigmoid")(x)

    return Model(inputs, x)

In [12]:
import os
import numpy as np
import cv2
from glob import glob
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger, TensorBoard
from concurrent.futures import ThreadPoolExecutor, wait, ALL_COMPLETED

In [13]:
def iou(y_true, y_pred):
     def f(y_true, y_pred):
         intersection = (y_true * y_pred).sum()
         union = y_true.sum() + y_pred.sum() - intersection
         x = (intersection + 1e-15) / (union + 1e-15)
         x = x.astype(np.float64)
         return x
     return tf.numpy_function(f, [y_true, y_pred], tf.float64)

In [23]:
batch = 32
lr = 1e-4
epochs = 300

In [24]:
train_dataset = tf_dataset(train_x, train_y, batch=batch)
valid_dataset = tf_dataset(valid_x, valid_y, batch=batch)

In [25]:
model = build_model()

In [26]:
opt = tf.keras.optimizers.Adam(lr)               
metrics = ["acc", iou]
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=metrics)

In [27]:
save_path = r'E:\GitHub\AutoDoc\NN_2\result_model'

In [28]:
callbacks = [
        ModelCheckpoint(os.path.join(save_path,"model.h5")),
        ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4),
        CSVLogger(os.path.join(save_path,"data_log.csv")),
        TensorBoard(),
        EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
    ]

In [29]:
train_steps = len(train_x)//batch
valid_steps = len(valid_x)//batch

if len(train_x) % batch != 0:
    train_steps += 1
if len(valid_x) % batch != 0:
  valid_steps += 1

print(train_steps)
print(valid_steps)

78
20


In [30]:
model.fit(train_dataset,
        validation_data=valid_dataset,
        epochs=epochs,
        steps_per_epoch = train_steps,
        validation_steps = valid_steps,
        callbacks=callbacks)

Epoch 1/300
78/78 [==============================] - 24s 285ms/step - loss: 0.4208 - acc: 0.8221 - iou: 0.4984 - val_loss: 0.5941 - val_acc: 0.6641 - val_iou: 0.2368 - lr: 1.0000e-04
Epoch 2/300
78/78 [==============================] - 18s 224ms/step - loss: 0.2810 - acc: 0.8899 - iou: 0.6133 - val_loss: 0.5510 - val_acc: 0.6641 - val_iou: 0.2244 - lr: 1.0000e-04
Epoch 3/300
78/78 [==============================] - 19s 243ms/step - loss: 0.2366 - acc: 0.9078 - iou: 0.6511 - val_loss: 0.5396 - val_acc: 0.6668 - val_iou: 0.2065 - lr: 1.0000e-04
Epoch 4/300
78/78 [==============================] - 19s 241ms/step - loss: 0.2154 - acc: 0.9151 - iou: 0.6707 - val_loss: 0.3933 - val_acc: 0.8391 - val_iou: 0.3540 - lr: 1.0000e-04
Epoch 5/300
78/78 [==============================] - 20s 251ms/step - loss: 0.1963 - acc: 0.9235 - iou: 0.6889 - val_loss: 0.2271 - val_acc: 0.9224 - val_iou: 0.5696 - lr: 1.0000e-04
Epoch 6/300
78/78 [==============================] - 18s 224ms/step - loss: 0.1734 - 

KeyboardInterrupt: 